# 랜덤 포레스트 분류 모델

In [0]:
import pandas as pd
import numpy as np
import os

In [0]:
DATA_IN_PATH = r'./data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)
reviews = list(train_data['review'])
y = np.array(train_data['sentiment'])

# CountVectorizer를 활용한 벡터화

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word', max_features=5000)

train_data_features = vectorizer.fit_transform(reviews)

# 학습과 검증 데이터셋 분리

In [0]:
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.2
RANDOM_SEED = 42

train_input, eval_input, train_label, eval_label = train_test_split(train_data_features, y,
                                                                    test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [0]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 분류기에 100개의 의사결정 트리를 사용한다
forest = RandomForestClassifier(n_estimators = 100)

# 단어 묶음을 벡터화한 데이터와 정답 데이터를 가지고 학습을 시작한다
forest.fit(train_input, train_label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# 검증 데이터셋으로 성능 평가

In [0]:
print("Accuracy: %f" % forest.score(eval_input, eval_label))

Accuracy: 0.846800


# 캐글에 데이터 제출

In [0]:
TEST_CLEAN_DATA = 'test_clean.csv'
DATA_OUT_PATH = r'./data_out/'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_reviews = list(test_data['review'])
ids = list(test_data['id'])

In [0]:
test_data_features = vectorizer.transform(test_reviews)

In [0]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# 위에서 만든 랜덤 포레스트 분류기를 통해 예측값을 가져온다
result = forest.predict(test_data_features)
# 판다스 데이너 프레임을 통해 데이터를 구성해서 output에 넣는다
output = pd.DataFrame(data={'id': ids, 'sentiment': result})
# csv파일을 생성
output.to_csv(DATA_OUT_PATH + "RandomForest_answer.csv", index=False, quoting=3)